scrape_the_salsePerson_name


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

# Set up Chrome WebDriver with Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# Navigate to the desired Amazon product page
product_url = "https://www.amazon.it/dp/B0B9HS2YCW"  # Replace with the desired product URL
driver.get(product_url)

# Extract the page source after the page is loaded
html = driver.page_source

# Close the Selenium WebDriver
driver.quit()

# Parse the page source using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the seller name from the Buy Box
seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

if seller_element is not None:
    seller_name = seller_element.get_text(strip=True)
    print("Seller Name:", seller_name)
else:
    print("Salesperson: Amazon")


    


Seller Name: SUPERBE BEBE®


scrape seller name for list of asins

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

def get_seller_names(asins):
    # Set up Chrome WebDriver with Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)

    seller_names = {}

    for asin in asins:
        # Navigate to the desired Amazon product page
        product_url = f"https://www.amazon.it/dp/{asin}"  # Use string formatting to replace {asin}
        driver.get(product_url)

        # Extract the page source after the page is loaded
        html = driver.page_source

        # Parse the page source using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find the seller name from the Buy Box
        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

        if seller_element is not None:
            seller_name = seller_element.get_text(strip=True)
            seller_names[asin] = seller_name
        else:
            seller_names[asin] = "Amazon"

    # Close the Selenium WebDriver
    driver.quit()

    return seller_names

# Example usage
asins = ['B0B9HS2YCW', 'B072J2J6GZ', 'B07DM7DWJF']
result = get_seller_names(asins)

# Print the seller names
for asin, seller_name in result.items():
    print(f"ASIN: {asin}, Seller Name: {seller_name}")


ASIN: B0B9HS2YCW, Seller Name: SUPERBE BEBE®
ASIN: B072J2J6GZ, Seller Name: Amazon
ASIN: B07DM7DWJF, Seller Name: ALENYK


scrape sellername for asin read from csv and output results in csv


In [ ]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup

def get_seller_names(asins):
    # Set up Chrome WebDriver with Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)

    seller_names = {}

    for asin in asins:
        # Navigate to the desired Amazon product page
        product_url = f"https://www.amazon.it/dp/{asin}"  # Use string formatting to replace {asin}
        driver.get(product_url)

        # Extract the page source after the page is loaded
        html = driver.page_source

        # Parse the page source using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find the seller name from the Buy Box
        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

        if seller_element is not None:
            seller_name = seller_element.get_text(strip=True)
            seller_names[asin] = seller_name
        else:
            seller_names[asin] = "Amazon"

    # Close the Selenium WebDriver
    driver.quit()

    return seller_names

# Read ASINs from CSV file
def read_asins_from_csv(filename):
    asins = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            asins.extend(row)
    return asins

# Write seller names to CSV file
def write_seller_names_to_csv(filename, seller_names):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name'])
        for asin, seller_name in seller_names.items():
            csv_writer.writerow([asin, seller_name])

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa Growers/buyboxMonitor/uniqueInputs.csv'
output_csv = '/Users/asifa/Desktop/Asifa Growers/buyboxMonitor/outputnew.csv'
asin_column_name = 'ASIN'  # Specify the column name for ASINs in the CSV

asins = read_asins_from_csv(input_csv)
seller_names = get_seller_names(asins)
write_seller_names_to_csv(output_csv, seller_names)


scrape sellername for asin also the availabilty ,and page not found (final code)

In [ ]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup

def get_seller_names_and_availability(asins):
    # Set up Chrome WebDriver with Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)

    seller_data = []

    for asin in asins:
        # Navigate to the desired Amazon product page
        product_url = f"https://www.amazon.it/dp/{asin}"  # Use string formatting to replace {asin}
        driver.get(product_url)

        # Extract the page source after the page is loaded
        html = driver.page_source

        # Parse the page source using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find the seller name from the Buy Box
        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

        

        # Find the availability <div> element
        availability_div = soup.find('div', {'id': 'availability'})

        if availability_div is not None:
            # Check if the product is available or not
            if "Non disponibile" in availability_div.get_text():
                availability_status = "Not Available"
                seller_name ="none"
            else:
                availability_status = "Available"
                if seller_element is not None:
                    seller_name = seller_element.get_text(strip=True)
                else:
                    seller_name = "Amazon"





        else:
            availability_status = "Product page not found"
            seller_name="none"

        seller_data.append([asin, seller_name, availability_status])

    # Close the Selenium WebDriver
    driver.quit()

    return seller_data

# Read ASINs from CSV file
def read_asins_from_csv(filename):
    asins = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            asins.extend(row)
    return asins

# Write seller names and availability to CSV file
def write_seller_names_and_availability_to_csv(filename, seller_data):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name', 'Availability_of_Product'])
        for data in seller_data:
            csv_writer.writerow(data)

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa Growers/buyboxMonitor/uniqueInputs.csv'
output_csv = '/Users/asifa/Desktop/Asifa Growers/buyboxMonitor/finalResult.csv'
asin_column_name = 'ASIN'  # Specify the column name for ASINs in the CSV

asins = read_asins_from_csv(input_csv)
seller_data = get_seller_names_and_availability(asins)
write_seller_names_and_availability_to_csv(output_csv, seller_data)


scrapping #other_seller,their names and their price


In [ ]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup

def get_seller_data(asins):
    # Set up Chrome WebDriver with Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)

    seller_data = {}

    for asin in asins:
        # Navigate to the desired Amazon product page
        product_url = f"https://www.amazon.it/dp/{asin}"
        driver.get(product_url)

        # Extract the page source after the page is loaded
        html = driver.page_source

        # Parse the page source using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find the seller name from the Buy Box
        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

        if seller_element is not None:
            seller_name = seller_element.get_text(strip=True)
        else:
            seller_name = "Amazon"

        # Find the number of other sellers
        num_other_sellers_element = soup.find('input', {'id': 'aod-total-offer-count'})
        num_other_sellers = int(num_other_sellers_element['value']) if num_other_sellers_element else 0

        # Find seller names and prices for other sellers
        other_sellers_list = []
        if num_other_sellers > 0:
            other_sellers = soup.find_all('div', {'class': 'aod-offer'})
            for seller in other_sellers:
                seller_name = seller.find('span', {'class': 'a-offscreen'}).text.strip()
                seller_price = seller.find('span', {'class': 'a-price'}).find('span', {'class': 'a-offscreen'}).text.strip()
                other_sellers_list.append({"seller_name": seller_name, "price": seller_price})

        seller_data[asin] = {"seller_name": seller_name, "other_sellers": other_sellers_list}

    # Close the Selenium WebDriver
    driver.quit()

    return seller_data

# Write seller data to CSV file
def write_seller_data_to_csv(filename, seller_data):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name', 'Number_of_other_sellers', 'Seller List'])
        for asin, data in seller_data.items():
            seller_name = data["seller_name"]
            other_sellers = data["other_sellers"]
            num_other_sellers = len(other_sellers)
            seller_list = ", ".join([f"{seller['seller_name']} ({seller['price']})" for seller in other_sellers])
            csv_writer.writerow([asin, seller_name, num_other_sellers, seller_list])

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa Growers/buyboxMonitor/input2.csv'
output_csv = '/Users/asifa/Desktop/Asifa Growers/buyboxMonitor/output2.csv'

asins = read_asins_from_csv(input_csv)
seller_data = get_seller_data(asins)
write_seller_data_to_csv(output_csv, seller_data)   


Failed to retrieve product details.


Salesperson not found
